In [ ]:
import simpy
import numpy as np

# Parámetros globales
N = 5  # Número de ONUs
R = 1_000  # Tasa total de transmisión (Mbps)
T_cycle = 2  # Longitud del ciclo de polling (ms)
T_guard = 0.1  # Tiempo de resguardo (ms)
W = [0.3, 0.2, 0.1, 0.2, 0.2]  # SLA de cada ONU (debe sumar 1)

# Calcular B_min para cada ONU (constante para cada ONU)
B_min = [(T_cycle - 2 * N * T_guard) * W[i] * R / (8 * sum(W)) for i in range(N)]

# Clase ONU
class ONU:
    def __init__(self, id, avg_rate, b_min):
        self.id = id
        self.avg_rate = avg_rate  # Tasa promedio de generación (paquetes/ms)
        self.ef_request = 0  # Solicitudes para EF
        self.af_request = 0  # Solicitudes para AF
        self.be_request = 0  # Solicitudes para BE
        self.ef_grant = 0  # Asignación EF
        self.af_grant = 0  # Asignación AF
        self.be_grant = 0  # Asignación BE
        self.b_min = b_min  # B_min calculado previamente
        self.b_min_prime = b_min  # B_min' dinámico

    def generate_requests(self, cycle_factor=1.0):
        self.ef_request = np.random.poisson(self.avg_rate * 0.2 * cycle_factor)
        self.af_request = np.random.poisson(self.avg_rate * 0.5 * cycle_factor)
        self.be_request = np.random.poisson(self.avg_rate * 0.3 * cycle_factor)

# Clase OLT
class OLT:
    def __init__(self, env, num_onus, cycle_time, total_bandwidth, guard_time):
        self.env = env
        self.num_onus = num_onus
        self.cycle_time = cycle_time
        self.total_bandwidth = total_bandwidth
        self.guard_time = guard_time
        self.onus = []  # Lista de ONUs
        
    def calculate_bandwidth_utilization(self):
        """Calcula la utilización del ancho de banda en el ciclo actual."""
        total_allocated = sum(
            onu.ef_grant + onu.af_grant + onu.be_grant for onu in self.onus
        )
        return (total_allocated / self.total_bandwidth) * 100
    
    def calculate_normalized_load(self):
        """Calcula la carga normalizada en el ciclo actual."""
        total_requests = sum(
            onu.ef_request + onu.af_request + onu.be_request for onu in self.onus
        )
        return total_requests / self.total_bandwidth

    def allocate_bandwidth(self, cycle_count):
        
        """Asignar ancho de banda basado en prioridades."""
        cycle_factor = 0.5 + 0.1 * (cycle_count % 10)  # Escalar entre 50% y 150%
        
        onu.generate_requests(cycle_factor=cycle_factor)
    
        # Comprobar si el sistema está en baja o alta carga
        print(onu.af_request + onu.be_request, onu.b_min_prime)
        if onu.af_request + onu.be_request <= onu.b_min_prime:
            
            # Baja carga
            onu.ef_grant = onu.ef_request  # G^H_i = H_i
            onu.af_grant = onu.af_request  # G^M_i = M_i
            onu.be_grant = onu.be_request  # G^L_i = L_i
        else:
            # Alta carga
            # Paso 1: Calcular g_i^H (EF)
            onu.ef_grant = onu.ef_request  # g_i^H = H_i
            onu.b_min_prime = max(0, onu.b_min - onu.ef_grant)  # Actualizar B_min'
    
            # Paso 2: Calcular g_i^M (AF) y g_i^L (BE)
            onu.af_grant = min(onu.af_request, onu.b_min_prime)  # g_i^M
            onu.be_grant = max(0, onu.b_min_prime - onu.af_grant)  # g_i^L
    
            # Paso 3: Ajustar G_i^M y G_i^L según la fórmula
            onu.af_grant = min(onu.af_request + onu.b_min_prime - onu.ef_grant, onu.b_min_prime)  # G^M_i
            onu.be_grant = max(0, onu.b_min - onu.ef_grant - min(onu.af_request, onu.b_min_prime))  # G^L_i
    
    def run(self):
        cycle_count = 0
        results = []  # Guardar datos para análisis
    
        while True:
            # Generar solicitudes y asignar ancho de banda
            for onu in self.onus:
                self.allocate_bandwidth(cycle_count)
    
            # Calcular métricas
            normalized_load = self.calculate_normalized_load()
            utilization = self.calculate_bandwidth_utilization()
    
            # Guardar resultados
            results.append((normalized_load, utilization))
    
            # Imprimir resultados
            print(f"Cycle {cycle_count}: Load={normalized_load:.2f}, Utilization={utilization:.2f}%")
            for onu in self.onus:
                print(f"ONU {onu.id}: EF={onu.ef_request}/{onu.ef_grant:.2f}, "
                      f"AF={onu.af_request}/{onu.af_grant:.2f}, "
                      f"BE={onu.be_request}/{onu.be_grant:.2f}, "
                      f"B_min'={onu.b_min_prime:.2f}")
            
    
            cycle_count += 1
            yield self.env.timeout(self.cycle_time)
    
        return results
# Crear entorno y OLT
env = simpy.Environment()
olt = OLT(env, N, T_cycle, R, T_guard)

# Crear ONUs y agregarlas al OLT
for i in range(N):
    onu = ONU(i, avg_rate=100, b_min=B_min[i])  # Tasa promedio de generación (paquetes/ms)
    olt.onus.append(onu)

# Iniciar simulación
env.process(olt.run())
env.run(until=1)  # Simular 10 ms